In [ ]:
!pip install transformers pillow

In [ ]:
!pip install bing-image-downloader

from bing_image_downloader import downloader

queries = ["pen", "ball", "cup", "book", "phone", "bottle", "shoes", "chair", "laptop", "table"]

for q in queries:
    downloader.download(q, limit=5, output_dir="dataset", adult_filter_off=True, force_replace=False)


In [ ]:
!unzip dataset.zip #only use if you upload dataset zip file in this

In [ ]:
!pip install torch torchvision transformers

In [ ]:
from transformers import CLIPProcessor, CLIPModel
import torch
import os, glob
from PIL import Image
from torch.nn.functional import cosine_similarity
from IPython.display import display

model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

model_path = "clip_embedding.pth"


if os.path.exists(model_path):
    print("Found saved embeddings")
    embeddings = torch.load(model_path, map_location="cpu")

else:
    print("No embeddings found, computing...")
    embeddings = {}
    image_paths = glob.glob("dataset/*/*.jpg", recursive=True)
    for path in image_paths:
        image = Image.open(path).convert("RGB")
        inputs = processor(images=image, return_tensors="pt")
        with torch.no_grad():
            img_emb = model.get_image_features(**inputs)
        embeddings[path] = img_emb
    torch.save(embeddings, model_path)
    print("Embeddings saved!")

def search_images(query, top_k=3):
    inputs = processor(text=query, return_tensors="pt")
    with torch.no_grad():
      text_emb = model.get_text_features(**inputs)

    scores = []
    for path, img_emb in embeddings.items():
        score = cosine_similarity(text_emb, img_emb).item()
        scores.append((path, score))

    scores = sorted(scores, key=lambda x: x[1], reverse=True)
    return scores[:top_k]

query = input("Enter search query: ")
results = search_images(query, top_k=3)

for path, score in results:
    print(f"{path} (score: {score:.4f})")
    display(Image.open(path))
